In [40]:
#imports

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import pathlib
import gc

In [41]:
class_names = ['NonDemented', 'Demented']
num_classes = len(class_names)
batch_size = 32
input_shape = (176, 208, 3)

def process(image, label):
    image /= 255.
    label = tf.one_hot(label, num_classes)
    return image, label

def flip(image, label):
    x = tf.image.random_flip_left_right(image)
    images = [image, x]
    labels = [label, label]
    return images, labels

def augment(image, label):
    func = tf.numpy_function(flip, [image, label], [tf.float32, tf.float32])
    return func

In [10]:
train_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/train')
test_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/test')

train_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        labels='inferred',
        class_names=class_names,
        batch_size=batch_size,
        image_size=input_shape[0:2])

test_dataset = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        labels='inferred',
        class_names=class_names,
        batch_size=batch_size,
        image_size=input_shape[0:2])

_train_dataset = train_dataset.map(process, num_parallel_calls=tf.data.experimental.AUTOTUNE)
_train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE).unbatch().unbatch()
test_dataset = test_dataset.map(process, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_x = np.concatenate([x for x, y in _train_dataset])
train_y = np.concatenate([y for x, y in _train_dataset])
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_dataset = train_dataset.batch(batch_size)

fig = plt.figure(figsize=(20,20))

batch_count = 0
image_count = 0
for images, labels in train_dataset:
    if batch_count == 0:
        print(np.shape(images))
        print(np.shape(labels))
        for i in range(np.shape(labels)[0]):
            image = images[i]
            fig.add_subplot(8,8,i+1)
            plt.imshow(image)
    batch_count += 1
    image_count += np.shape(labels)[0]
   
plt.show()

#image_count = 0
#for image, label in train_dataset:
#    if image_count < 32:
#        fig.add_subplot(8,8,image_count+1)
#        plt.imshow(image)
#    image_count += 1
   
#plt.show()

print(tf.data.experimental.cardinality(train_dataset))
#print(f"{image_count} images")
print(f"{batch_count} batches, {image_count} images")

Found 5121 files belonging to 2 classes.
Found 1279 files belonging to 2 classes.


InvalidArgumentError: 1-th value returned by pyfunc_3 is int32, but expects float
	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

In [42]:
######################## deprecated ########################

train_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/train')
test_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/test')

datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
                                                       #rotation_range=10,
                                                       rescale=1./255
                                                       #brightness_range=(0.8,1.1),
                                                       zoom_range=0.2,
                                                       #width_shift_range=0.1,
                                                       #height_shift_range=0.1
                                                      )

simple_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
        train_dir,
        classes=class_names,
        class_mode='categorical',
        shuffle=True,
        batch_size=batch_size,
        target_size=input_shape[0:2])

test_generator = simple_datagen.flow_from_directory(
        test_dir,
        classes=class_names,
        class_mode='categorical',
        shuffle=True,
        batch_size=batch_size,
        target_size=input_shape[0:2])

SyntaxError: invalid syntax (Temp/ipykernel_15292/2426998501.py, line 10)

In [43]:
vgg = keras.applications.VGG19(
        include_top=False,
        input_shape=input_shape,
        pooling=max)

for layer in vgg.layers:
    layer.trainable = False
#vgg.layers[17].trainable = True
#vgg.layers[18].trainable = True
#vgg.layers[19].trainable = True
#vgg.layers[20].trainable = True

vgg.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 176, 208, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 176, 208, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 176, 208, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 88, 104, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 88, 104, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 88, 104, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 44, 52, 128)       0     

In [48]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.02, 10)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("alzheimer_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [50]:
#### Create a transfer learning model ####

epochs = 30
keras.backend.clear_session()
gc.collect()
model = keras.models.Sequential([
        keras.Input(input_shape),
        layers.GaussianNoise(0.08),
        vgg,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.7),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax'),
    
    ])

model.summary()
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=[tf.keras.metrics.AUC(name='auc')])
history = model.fit(train_generator,
                    #steps_per_epoch=image_count // batch_size,
                    batch_size=batch_size,
                    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler],
                    epochs=epochs,
                    validation_data=test_generator
                    #validation_steps=800 // batch_size
                   )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise (GaussianNois (None, 176, 208, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 5, 6, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 15360)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               7864832   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               6

In [ ]:
#### Create a CNN from scratch ####

epochs = 30
keras.backend.clear_session()
gc.collect()
model = keras.models.Sequential([
        keras.Input(input_shape),
        #layers.GaussianNoise(0.5),
        #Creates the convolutional layer
        layers.BatchNormalization(),
        layers.Conv2D(16, kernel_size=(3,3), activation='relu'),
        layers.Conv2D(16, kernel_size=(3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        
        layers.SeparableConv2D(32, kernel_size=(3,3), activation='relu'),
        layers.SeparableConv2D(32, kernel_size=(3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        
        layers.SeparableConv2D(64, kernel_size=(3,3), activation='relu'),
        layers.SeparableConv2D(64, kernel_size=(3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        
        layers.SeparableConv2D(128, kernel_size=(3,3), activation='relu'),
        layers.SeparableConv2D(128, kernel_size=(3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        
        layers.SeparableConv2D(256, kernel_size=(3,3), activation='relu'),
        layers.SeparableConv2D(256, kernel_size=(3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
    
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(num_classes, activation='softmax'),
    ])
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
history = model.fit(train_generator, batch_size=batch_size, epochs=epochs, validation_data=test_generator)

In [ ]:
plt.title('VGG19 Transfer Learning w/ Gaussian(1) Accuracy with Dropout')

plt.plot(history.history['accuracy'], label = "training acc.")
plt.plot(history.history['val_accuracy'], label = "validation acc.")
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training acc.', 'validation acc.'], loc='upper left')

plt.savefig("../Project/Result2/VGG19_MODEL_Pre_1_ACCURACY_adam_dropout.png")
plt.show()
